# AutoGluon

## Setup
### Correct the path

In [1]:
import sys
import os

# Get the current working directory
current_working_directory = os.getcwd()

# Go up one level from the current working directory
parent_directory = os.path.join(current_working_directory, '..')

# Add the parent directory to sys.path
sys.path.append(parent_directory)

os.getcwd()

'/home/igorai/projects/deloitte/task5/notebooks'

In [2]:
%pip install autogluon
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
%load_ext autoreload

In [4]:
%autoreload 

from sklearn.metrics import accuracy_score, classification_report
import autogluon.core as ag
from autogluon.tabular import TabularDataset, TabularPredictor
from datetime import datetime

from src.features.post_processor import save_predictions
from src.features.ml_service import  prepare_data, prepare_test_data
from src.data.data_loader import load_test_raw_data

## Load data

In [5]:
x_train, x_val, x_test, y_train, y_val, y_test = prepare_data()
target_feature_name = "IsFraud"
x_train[target_feature_name] = y_train  # Add the target column back to your training data

data = TabularDataset(x_train)

## Train model

In [6]:
# Initialize the AutoGluon TabularPredictor
target_feature_name = "IsFraud"
metric = "f1"
predictor = TabularPredictor(label=target_feature_name, eval_metric=metric).fit(train_data=data)


No path specified. Models will be saved in: "AutogluonModels/ag-20240316_140013"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240316_140013"
=================== System

## Make predictions

In [7]:

# Evaluate on the validation set
y_validate_pred = predictor.predict(x_val)
valid_accuracy = accuracy_score(y_val, y_validate_pred)
print("Validation Accuracy: ", valid_accuracy)

# Evaluate on the test set
y_test_pred = predictor.predict(x_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy: ", test_accuracy)
print("Test Classification Report:\n", classification_report(y_test, y_test_pred))

Validation Accuracy:  0.9162875
Test Accuracy:  0.9134875
Test Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.97      0.95     74379
           1       0.24      0.10      0.15      5621

    accuracy                           0.91     80000
   macro avg       0.59      0.54      0.55     80000
weighted avg       0.89      0.91      0.90     80000



## Save model

In [14]:

x_test = prepare_test_data()
final_predictions = predictor.predict(x_test)
# 
x_test = load_ra
x_test["IsFraud"] = final_predictions
final_predictions = x_test[["TransactionID", "IsFraud"]]

KeyError: "['TransactionID'] not in index"

In [9]:
# # Save the final predictions as a CSV file
current_time = datetime.now().strftime("%Y%m%d%H%M%S")
save_predictions(final_predictions, f'predictions_auto_gluon_{current_time}.csv')